In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Generate synthetic sine wave dataset
class SineWaveDataset(Dataset):
    def __init__(self, seq_length, num_samples):
        self.seq_length = seq_length
        self.num_samples = num_samples
        self.data = []
        self.labels = []
        
        # Generate sine wave data
        t = np.linspace(0, 100, num_samples + seq_length + 1)
        sine_wave = np.sin(t)
        
        for i in range(num_samples):
            x = sine_wave[i:i+seq_length]
            y = sine_wave[i+seq_length]
            self.data.append(x)
            self.labels.append(y)
            
        self.data = torch.FloatTensor(self.data).reshape(-1, seq_length, 1)
        self.labels = torch.FloatTensor(self.labels).reshape(-1, 1)
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Define RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate RNN
        out, _ = self.rnn(x, h0)
        
        # Get output from the last time step
        out = self.fc(out[:, -1, :])
        return out

# Hyperparameters
seq_length = 10
hidden_size = 64
num_layers = 2
num_epochs = 100
batch_size = 32
learning_rate = 0.001
num_samples = 1000

# Create datasets and dataloaders
train_dataset = SineWaveDataset(seq_length, num_samples)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function, and optimizer
model = RNN(input_size=1, hidden_size=hidden_size, num_layers=num_layers, output_size=1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
train_losses = []
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}')

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.grid(True)
plt.savefig('training_loss.png')
plt.show()  # Added to display the plot
plt.close()

# Generate predictions for visualization
model.eval()
test_dataset = SineWaveDataset(seq_length, 100)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

predictions = []
actuals = []
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        predictions.extend(outputs.cpu().numpy().flatten())
        actuals.extend(batch_y.cpu().numpy().flatten())

# Plot predictions vs actual
plt.figure(figsize=(12, 6))
plt.plot(actuals, label='Actual', alpha=0.7)
plt.plot(predictions, label='Predicted', alpha=0.7)
plt.title('RNN Predictions vs Actual Values')
plt.xlabel('Sample')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.savefig('predictions.png')
plt.show()  # Added to display the plot
plt.close()

# Print final loss
print(f'Final Training Loss: {train_losses[-1]:.6f}')

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load and preprocess the dataset
class ClimateDataset(Dataset):
    def __init__(self, data, seq_length, target_col='meantemp'):
        self.seq_length = seq_length
        self.target_col = target_col
        
        # Normalize the data
        self.scaler = MinMaxScaler()
        data_normalized = self.scaler.fit_transform(data[[target_col]])
        
        self.data = []
        self.labels = []
        
        # Create sequences
        for i in range(len(data_normalized) - seq_length):
            x = data_normalized[i:i+seq_length]
            y = data_normalized[i+seq_length]
            self.data.append(x)
            self.labels.append(y)
            
        self.data = torch.FloatTensor(self.data)
        self.labels = torch.FloatTensor(self.labels)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Define RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate RNN
        out, _ = self.rnn(x, h0)
        
        # Get output from the last time step
        out = self.fc(out[:, -1, :])
        return out

# Load the dataset (assuming it's in Kaggle input directory)
try:
    df = pd.read_csv('/kaggle/input/daily-climate-time-series-data/DailyDelhiClimateTrain.csv')
except FileNotFoundError:
    print("Please ensure the Daily Delhi Climate dataset is available in the Kaggle input directory.")
    exit()

# Convert date to datetime and sort
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

# Hyperparameters
seq_length = 10
hidden_size = 64
num_layers = 2
num_epochs = 100
batch_size = 32
learning_rate = 0.001

# Create dataset and dataloader
dataset = ClimateDataset(df, seq_length, target_col='meantemp')
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, loss function, and optimizer
model = RNN(input_size=1, hidden_size=hidden_size, num_layers=num_layers, output_size=1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
train_losses = []
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}')

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.grid(True)
plt.savefig('training_loss.png')
plt.show()  # Added to display the plot
plt.close()

# Generate predictions for visualization
model.eval()
predictions = []
actuals = []
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        
        # Inverse transform to original scale
        outputs = dataset.scaler.inverse_transform(outputs.cpu().numpy())
        batch_y = dataset.scaler.inverse_transform(batch_y.cpu().numpy())
        
        predictions.extend(outputs.flatten())
        actuals.extend(batch_y.flatten())

# Plot predictions vs actual
plt.figure(figsize=(12, 6))
plt.plot(actuals, label='Actual Temperature', alpha=0.7)
plt.plot(predictions, label='Predicted Temperature', alpha=0.7)
plt.title('RNN Temperature Predictions vs Actual Values')
plt.xlabel('Sample')
plt.ylabel('Mean Temperature (°C)')
plt.legend()
plt.grid(True)
plt.savefig('predictions.png')
plt.show()  # Added to display the plot
plt.close()

# Print final loss
print(f'Final Training Loss: {train_losses[-1]:.6f}')